In [1]:
!pip install llama-cpp-python
!wget https://huggingface.co/lmstudio-community/Llama-3.2-1B-Instruct-GGUF/resolve/main/Llama-3.2-1B-Instruct-Q8_0.gguf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=3485338 sha256=24c7fc76af163a21e5a631f62d0b626c44544687b682d905c2f28f6b26930bb6
  Stored in directory: /root/.cache/pip/wheels/f8/b0/a2/f47d952aec7ab061b9e2a345e23a1e1e137beb7891259e3d0c
Successfully built llama-cpp-python
--2024-10-26 13:57:41--  https://huggingface.co/lmstudio-community/Llama-3.2-1B-Instruct-GGUF/resolve/main/Llama-3.2-1B-Instruct-Q8_0.gguf
Resolving huggingface.co (huggingface.co)... 3.169.36.71, 3.169.36.25, 3.169.36.98, ...
Connecting to huggingface.co (huggingface.co)|3.169.36.71|:443... connected.
HTTP request sent, awaiting resp

In [2]:
from llama_cpp import Llama
import pathlib
import pandas as pd
import math

#Defining models
MODEL_Q8_0 = Llama(
    model_path="Llama-3.2-1B-Instruct-Q8_0.gguf",
    n_ctx=128, n_gpu_layers=128, logits_all=True
)

#Defining function for getting a response
def query_with_logprobs(model, question):
    prompt = f"Q: {question} A:"
    output = model(prompt=prompt, max_tokens=1, temperature=10000, logprobs=True)
    response = output["choices"][0]
    text = response["text"].strip().upper()
    print(response["logprobs"]["top_logprobs"])
    logprobs = response["logprobs"]["top_logprobs"][0]  # Get logprobs for first token

    # Extract logprobs for TRUE and FALSE
    logprob_true = math.exp(logprobs.get(" TRUE", float("-inf")))
    logprob_false = math.exp(logprobs.get(" FALSE", float("-inf")))
    print(logprob_true, logprob_false)
    return text, logprob_true, logprob_false

#Resolve Wikidata IDs to English labels
df_items = pd.read_excel("Classeur1.xlsx", sheet_name="Items")
df_props = pd.read_excel("Classeur1.xlsx", sheet_name="Properties")
l_items = dict(zip(df_items["ID"], df_items["Label"]))
l_props = dict(zip(df_props["ID"], df_props["Label"]))
df_verify = pd.read_excel("ClinMed.xlsx", sheet_name="To Add")

def get_item(x):
    return l_items.get(x, "____")

df_verify["subj"] = df_verify["subject_id"].apply(get_item)
df_verify["obj"] = df_verify["object_id"].apply(get_item)

#Defining function to check a relation
def check_relation_with_llama(subject, object):
    response, logprob_true, logprob_false = query_with_logprobs(
        MODEL_Q8_0, f"Are the concepts '{subject}' and '{object}' directly related? Answer with one word: TRUE or FALSE."
    )
    return response, logprob_true, logprob_false

df_verify["llama_valid"] = ""  # Initialize a new column for LLAMA validity

for index, row in df_verify.iterrows():
    subject = row["subj"]
    object = row["obj"]

    if subject != "____" and object != "____":
        response, logprob_true, logprob_false = check_relation_with_llama(subject, object)

        # Calculate normalized score as a confidence metric based on logprobs
        score = (logprob_true) / (logprob_true + logprob_false)
        df_verify.at[index, "llama_valid"] = score
        print(score)

df_verify.to_excel("verify_with_llama_add_logprobs.xlsx", index=False)

llama_model_loader: loaded meta data with 31 key-value pairs and 147 tensors from Llama-3.2-1B-Instruct-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 1B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 1B
llama_model_loader: - kv   6:                            general.license str              = llama3.2
llama_model_loader: - kv   7:                       

[{' TRUE': -0.24456787, ' FALSE': -1.6700592}]
0.7830428405877424 0.18823592097026215
0.8061978410108366


Llama.generate: 13 prefix-match hit, remaining 23 prompt tokens to eval
llama_perf_context_print:        load time =    2306.15 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    23 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1557.38 ms /    24 tokens


[{' TRUE': -0.20992386}]
0.8106459634392621 0.0
1.0


Llama.generate: 13 prefix-match hit, remaining 27 prompt tokens to eval
llama_perf_context_print:        load time =    2306.15 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    27 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1776.05 ms /    28 tokens


...

[{' TRUE': -0.25041705}]
0.7784760490441271 0.0
1.0


Llama.generate: 12 prefix-match hit, remaining 19 prompt tokens to eval
llama_perf_context_print:        load time =    2306.15 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    19 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1372.71 ms /    20 tokens


[{' TRUE': -0.21404938}]
0.8073085159161731 0.0
1.0


Llama.generate: 12 prefix-match hit, remaining 18 prompt tokens to eval


KeyboardInterrupt: 